<a href="https://colab.research.google.com/github/Emmazch22/Paula_y_Emmanuel_CI0134/blob/main/english_dataset_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 5.3 MB/s eta 0:00:00


In [27]:
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
from unidecode import unidecode
import re

%matplotlib inline

In [5]:
# Se carga la libreria drive de google para cargar los archivos almacenados en drive
drive.mount("/content/gdrive")

# El dataset en cuestión contiene datos sobre tweets en diferentes idiomas, pero nos interesa utilizar solo
# los que se encuentran en ingles.
tweets = pd.read_csv('/content/gdrive/My Drive/tweets.csv').query('Language == "en"')
tweets = tweets[(tweets['Label'] == 'positive') | (tweets['Label'] == 'negative')]

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
tweets.head(10)

,Text,Language,Label
3,Rwanda is set to host the headquarters of Unit...,en,positive
5,It sucks for me since I'm focused on the natur...,en,negative
8,Social Security. Constant political crises dis...,en,negative
9,@FilmThePoliceLA A broken rib can puncture a l...,en,negative
10,@jacobringenwald @aKeithWatts @CountDankulaTV ...,en,negative
11,@NZ_HK_SU @telebusiness My question was rhetor...,en,negative
12,Wimbledon: Nick Kyrgios admits spitting toward...,en,positive
13,is booktwt a thing? if so thats her and she sp...,en,positive
16,@roipaee @joe98100959 @Formula1game @redbullra...,en,negative
17,🤩 Power back on (1855749): North VI in Ladysm...,en,negative


In [7]:
tweets.shape

(492662, 3)

In [16]:
tweets = tweets.drop('Language', axis=1)

In [17]:
n = 55360 # 111720 // 2

positive = tweets[tweets['Label'] == 'positive']
random_positive = positive.sample(n)
print(random_positive.head())
print(random_positive.shape)

                                                     Text     Label
445607  HotmarkPrint Revolution 1798 - OUR UNIVERSAL P...  positive
614679  There's still time to register for tomorrow's ...  positive
270362  planning to open pre order XH album for ina on...  positive
925420  READ. THIS. THREAD. IF. YOU. WANT. TO. IMPROVE...  positive
874748                   Charlie's dream 😂😂 #Supernatural  positive
(55360, 2)


In [18]:
negative = tweets[tweets['Label'] == 'negative']
random_negative = negative.sample(n)
print(random_negative.head())
print(random_negative.shape)

                                                     Text     Label
580618  @Pebnet @bpaliwal @http40415 @Noori00786 @Meme...  negative
694202  Trick question:\n\nJustice https://t.co/lmh0t2...  negative
576970  he speaks about his violent motorcycle acciden...  negative
778265                            Get lit 🔥 or Get Lost 😞  negative
3655         @myvillaneve "I blocked it out" poor ant 🤣🤣🤣  negative
(55360, 2)


In [23]:
concat_df = pd.concat([random_positive, random_negative])
print(concat_df.head(10))

                                                     Text     Label
445607  HotmarkPrint Revolution 1798 - OUR UNIVERSAL P...  positive
614679  There's still time to register for tomorrow's ...  positive
270362  planning to open pre order XH album for ina on...  positive
925420  READ. THIS. THREAD. IF. YOU. WANT. TO. IMPROVE...  positive
874748                   Charlie's dream 😂😂 #Supernatural  positive
144112  https://t.co/0G3lPoPC34 Lange boots video, is ...  positive
230710  @CyberFrogsNFT Excited to see a new competitor...  positive
282617  🍀🍀🍀🍀\n#WINNER #위너 #4thMINIALBUM #HOLIDAY #RELE...  positive
531602  Partner with Afya Intelligence and scale your ...  positive
357118  The @GUNNAROptiks Torpedo-X @Razer Edition is ...  positive


In [24]:
merged_df = concat_df.sample(frac=1).reset_index(drop=True)

In [25]:
print(merged_df.head(10))

                                                Text     Label
0  @salvatoshi From what I considered it seems th...  positive
1  Son, 14, who has all the time in the world to ...  positive
2                                            ur lost  negative
3  @newsmax It’s much easier to determine facts w...  positive
4    Asleep or awake, \nI dream of you all the same.  positive
5  @IsaiahArney @ProudSocialist .@IsaiahArney\nAn...  negative
6  boooo they are remodeling our office, and have...  negative
7  @ericksancan91 I can see where people say it's...  negative
8  @OMaraRules Oh dear, let's improve your sense ...  positive
9  @Mexicanblueeye @tRoseH2 @debbiecasey3 @Honest...  negative


In [26]:
merged_df.shape

(110720, 2)

In [28]:
def remove_emoji(string):
  emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', string) 

def clean_tweet(tweet):
    if type(tweet) == float:
        return ""
    temp = tweet.lower()
    temp = unidecode(temp)
    temp = re.sub("'", "", temp) # to avoid removing contractions in english
    temp = re.sub("@[A-Za-z0-9_ÑÁÉÍÓÚ]+","", temp)
    temp = re.sub("#","", temp)
    temp = remove_emoji(temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    temp = unidecode(temp)
    temp = re.sub("[^a-zA-Z0-9ñáéíóú]"," ", temp)
    return temp
    
## Una vez creadas las funciones se procede a realizar la limpieza de los tweets
merged_df['Text'] = merged_df['Text'].map(lambda x : clean_tweet(x))

In [29]:
merged_df.head(10)

,Text,Label
0,from what i considered it seems that adding s...,positive
1,son 14 who has all the time in the world to ...,positive
2,ur lost,negative
3,its much easier to determine facts when one s...,positive
4,asleep or awake i dream of you all the same,positive
5,and what what was your question im not a...,negative
6,boooo they are remodeling our office and have...,negative
7,i can see where people say its a party issue ...,negative
8,oh dear lets improve your sense of humor r...,positive
9,your ob sounds like an a hole but they st...,negative


In [30]:
merged_df.shape

(110720, 2)

In [31]:
# Montar el drive
drive.mount('/content/drive')

# Guardar el dataset en un archivo CSV en tu drive
merged_df.to_csv('/content/drive/My Drive/english_dataset.csv', index=True)

Mounted at /content/drive
